In [23]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim


ImportError: No module named Config

In [25]:

batch_size=10
embedding_filename = "glove_6B_200d.txt"
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]


In [9]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]



OSError: [Errno 2] No such file or directory: 'corpus/fulltext/'

In [10]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();




d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
print(docvec)


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [14]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [12]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [ ]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

words_data=pd.read_csv("words_data.csv");
catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

In [26]:
##sen2vec embeddings
def sen2Vec(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 58.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 108.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 158.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 218.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 268.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 338.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 398.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 438.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 468.00MB
INFO:tensorflow

In [ ]:
print(word_index['unk'])


In [18]:
def get_statements(file):
    with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    #catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    #sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    sentences+=tokenizer.tokenize(sentence.text)
    
    sentences = [w for w in sentences if not w in stop_words_list] 
    catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases


    

In [ ]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [17]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [19]:
sentences,catchphrases=get_statements(files[0]);
sentence_indexes,labels=lookup_indexes_labels(sentences,catchphrases)
print(len(sentence_indexes),len(labels))

sentence_index_matrix=prepare_index_matrices(sentence_indexes);
label_matrix=prepare_label_matrices(labels);

print(len(sentence_index_matrix),len(label_matrix))

NameError: name 'files' is not defined

In [20]:

class CatchPhraseExtractor(object):

    def __init__(self, graph):
        self.total_loss=0.0
        self.build_graph(graph)

    def build_graph(self, graph):
        print("generating graph");

        with graph.as_default():
            self.train_word_indices = tf.placeholder(tf.int32, shape=[None,5])
            self.train_word_labels = tf.placeholder(tf.int32, shape=[None,1])
            W = tf.Variable(word_embeddings, dtype=tf.float32) 
            phrase_tensor = tf.nn.embedding_lookup(W, self.train_word_indices)
            
            self.train_sentence_indices = tf.placeholder(tf.int32, shape=[None,1])
            #S = tf.Variable(sentence_embeddings, dtype=tf.float32)
            #sentence_vector = tf.nn.embedding_lookup(S, self.train_sentence_indices)
            sentence_tensor = tf.convert_to_tensor(np.array([sentence_embeddings[hash_val] for hash_val in self.train_sentence_indices])
                                                   
            self.train_document_vector = tf.placeholder(tf.int32, shape=[1,200])
            document_tensor = tf.convert_to_tensor(np.array([self.train_document_vector for _ in xrange(self.train_sentence_indices.shape[0])])
                     
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], 1000])
            conv_filter = tf.Variable(tf.truncated_normal((1000, 200), stddev=0.1));
            word_tensor = tf.nn.relu(tf.matmul(sentence_vectors, conv_filter));
            
            stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
                                                   
            weights_input = tf.Variable(tf.random_normal((600, 100), stddev=0.2));
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2));
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2));
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2));
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2);
            
            prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.train_labels,logits=self.sentence_values)
            self.loss=tf.reduce_mean(total_loss)
                               
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(0.1)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)

            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        h1 = tf.add(h1, bias1)
        h1 = tf.nn.tanh(h1)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        return tf.transpose(h2)
    
    
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
        
                                                   
    def train(self, sess, num_steps):
        self.init.run()
        print("Initailized")
        total_loss=0.0;
        for file in files[:1]:
            sentences,catchphrases=get_statements(file);
            
            sentence_indexes,labels=lookup_indexes_labels(sentences,catchphrases)
            
            sentence_index_matrix=prepare_index_matrices(sentence_indexes);
            label_matrix=prepare_label_matrices(labels);
            
            sentence_index_matrix=np.asarray(sentence_index_matrix)
            label_matrix=np.asarray(label_matrix)
        
            #sentence_indexes=np.reshape(sentence_indexes,(sentence_indexes.shape[0],1))
            label_matrix=np.reshape(label_matrix,(label_matrix.shape[0],1))
            
            
            feed_dict = {self.train_inputs: sentence_index_matrix, self.train_labels: label_matrix}
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            for i in range(len(outputs)):
                if label_matrix[i]==1.0:
                    print(outputs[i],label_matrix[i])
            print("Train Finished.",loss_val)
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [275]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [276]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph, embedding_array)


generating graph
Tensor("Max:0", shape=(1, 200), dtype=float32)


In [277]:
with tf.Session(graph=graph) as sess:
  
    model.load_embeddings(embedding_array)    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    global_step = tf.train.get_or_create_global_step()
    for i in range(1):
        model.train(sess, 10)   
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

Initailized
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [1.]
[0.466128] [